## Google colab setup

In [ ]:
try:
  import google.colab
  IS_COLAB = True
except:
  IS_COLAB = False

In [ ]:
if IS_COLAB:
  !pip install tensorflow_addons
  !pip install unidecode
  !pip install transformers
  # !pip install nlpaug

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 38.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 24.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 92.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 28.6 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/SemiSupervised/*.csv .

## Imports and setup

In [1]:
%%capture
import tensorflow as tf
import tensorflow_addons as tfa
import pandas as pd
import re
import time
import spacy
import numpy as np
from transformers import AutoTokenizer, TFAutoModel
from unidecode import unidecode

2023-04-29 14:14:42.175573: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-29 14:14:42.690241: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.1/lib64:/usr/local/cuda-10.2/lib64:/usr/local/cuda/extras/CUPTI/lib64::/usr/local/cuda-10.1/lib64:/usr/local/cuda-10.2/lib64:/usr/local/cuda/extras/CUPTI/lib64:
2023-04-29 14:14:42.690285: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object 

In [2]:
gpus = tf.config.list_physical_devices('GPU')
tf.config.set_visible_devices(gpus[1], 'GPU')

## Utils

In [3]:
data = pd.read_csv("train_ner.csv")

In [4]:
def preprocess(x):
    s = unidecode(x)
    s = str.lower(s)
    s = re.sub(r"\[[a-z]+\]","", s)
    s = re.sub(r"\*","", s)
    s = re.sub(r"[^a-zA-Z0-9]+"," ",s)
    s = re.sub(r" +"," ",s)
    s = re.sub(r"(.)\1+",r"\1",s)

    return s

In [5]:
label_ids = {label_name:i for i, label_name in enumerate(sorted(set(data["label"])))}
label_ids

{'ABUSE': 0, 'INSULT': 1, 'OTHER': 2, 'PROFANITY': 3}

In [6]:
data["preprocessed"] = data["text"].apply(preprocess)
data.head()

,id,text,label,preprocessed
0,32674,da de unde stii u mai [ORG] ca banii au fost p...,ABUSE,da de unde sti u mai ca bani au fost pt si nu ...
1,16514,m*uie [PERS] m*uie [PERS] ... m*uie\nbai kakat...,INSULT,muie muie muie bai kakatule stai in banca ta d...
2,32556,PT ALA CARE ARE TREABA CU [PERS]!!ESTI UNUL CA...,OTHER,pt ala care are treaba cu esti unul care nu ar...
3,23861,sunt bucuros ca [PERS] nu a mai venit la [ORG]...,OTHER,sunt bucuros ca nu a mai venit la jucator de d...
4,21811,[PERS] esti....PE..N\n ES..CU..LI..BI.L!!! te ...,INSULT,esti pe n es cu li bi l te asemeni cu de la c...


In [7]:
data["class"] = data["label"].map(lambda x: label_ids[x])
data.head(5)

,id,text,label,preprocessed,class
0,32674,da de unde stii u mai [ORG] ca banii au fost p...,ABUSE,da de unde sti u mai ca bani au fost pt si nu ...,0
1,16514,m*uie [PERS] m*uie [PERS] ... m*uie\nbai kakat...,INSULT,muie muie muie bai kakatule stai in banca ta d...,1
2,32556,PT ALA CARE ARE TREABA CU [PERS]!!ESTI UNUL CA...,OTHER,pt ala care are treaba cu esti unul care nu ar...,2
3,23861,sunt bucuros ca [PERS] nu a mai venit la [ORG]...,OTHER,sunt bucuros ca nu a mai venit la jucator de d...,2
4,21811,[PERS] esti....PE..N\n ES..CU..LI..BI.L!!! te ...,INSULT,esti pe n es cu li bi l te asemeni cu de la c...,1


In [8]:
data["label"].value_counts()

OTHER        3649
ABUSE        2768
INSULT       2242
PROFANITY    1294
Name: label, dtype: int64

## Prepare data splits

In [9]:
tok_robert = AutoTokenizer.from_pretrained("readerbench/RoBERT-base")
robert = TFAutoModel.from_pretrained("readerbench/RoBERT-base")

2023-04-29 14:14:48.285796: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-29 14:14:48.286374: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-29 14:14:48.286709: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-29 14:14:48.286992: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least on

In [10]:
def split_ssl_data(ids_array,mask_array,labels,num_classes,label_percent):
  labeled = None
  unlabeled = None

  for class_idx in range(num_classes):
    class_ids = ids_array[labels==class_idx]
    class_mask = mask_array[labels==class_idx]
    sz = int(label_percent * class_ids.shape[0])

    labels_reduced = labels[labels==class_idx][:sz]
    labeled_ids, unlabeled_ids = class_ids[:sz], class_ids[sz:]
    labeled_mask, unlabeled_mask = class_mask[:sz], class_mask[sz:]

    if not labeled:
      labeled = (labeled_ids, labeled_mask, labels_reduced)
      unlabeled = (unlabeled_ids, unlabeled_mask)
    else:
      labeled = (
          np.concatenate([labeled[0],labeled_ids]),
          np.concatenate([labeled[1],labeled_mask]),
          np.concatenate([labeled[2],labels_reduced])
      )
      unlabeled = (
          np.concatenate([unlabeled[0],unlabeled_ids]),
          np.concatenate([unlabeled[1],unlabeled_mask]),
      )

  return labeled, unlabeled


In [11]:
def preprocess_robert(x):
  t = tok_robert(x,padding="max_length",max_length=96,truncation=True,return_tensors='np')
  return t["input_ids"], t["attention_mask"]

def map_func(input_ids, masks, labels):
  return {'input_ids': input_ids, 'attention_mask': masks}, labels

def map_func2(input_ids, masks):
  return {'input_ids': input_ids, 'attention_mask': masks}

def prepare_ds(filename,batch_size=64):
  df = pd.read_csv(filename)
  X_id_mask = df['text'].map(preprocess).apply(preprocess_robert).apply(pd.Series)

  X_id_mask.columns = ["input_ids","attention_mask"]

  ids_array = np.squeeze(np.stack(X_id_mask.input_ids.values), axis=1)
  mask_array = np.squeeze(np.stack(X_id_mask.attention_mask.values), axis=1)
  labels = df["label"].map(lambda x: label_ids[x]).values

  res_ds = tf.data.Dataset.from_tensor_slices((ids_array, mask_array, labels)).map(map_func).shuffle(len(df)).batch(batch_size)
  return res_ds

In [12]:
def prepare_train_ds(filename,batch_size=16,label_percent=0.2):
  df = pd.read_csv(filename)
  df = df.sample(frac=1)
  X_id_mask = df['text'].map(preprocess).apply(preprocess_robert).apply(pd.Series)

  X_id_mask.columns = ["input_ids","attention_mask"]

  ids_array = np.squeeze(np.stack(X_id_mask.input_ids.values), axis=1)
  mask_array = np.squeeze(np.stack(X_id_mask.attention_mask.values), axis=1)
  labels = df["label"].map(lambda x: label_ids[x]).values

  labeled, unlabeled = split_ssl_data(ids_array,mask_array,labels,len(label_ids),label_percent)
  labeled_ds = tf.data.Dataset.from_tensor_slices(labeled)
  labeled_ds = labeled_ds.map(map_func).shuffle(len(labeled_ds)).batch(batch_size).repeat()
  unlabeled_ds = tf.data.Dataset.from_tensor_slices(unlabeled)
  unlabeled_ds = unlabeled_ds.map(map_func2).shuffle(len(unlabeled_ds)).batch(batch_size).repeat()
  
  return labeled_ds, unlabeled_ds

In [13]:
labeled_ds, unlabeled_ds = prepare_train_ds("train_ner.csv")
test_ds = prepare_ds("test_ner.csv")
val_ds = prepare_ds("validation_internal_ner.csv")

## Model definition and declaration

In [14]:
class FixMatchCR(tf.keras.Model):
  def __init__(self,bert_model,num_classes=4,**kwargs):
    super(FixMatchCR,self).__init__(**kwargs)
    self.bert = bert_model

    self.num_classes = num_classes
    self.weak_augment = tf.keras.layers.GaussianNoise(stddev=0.5)
    self.strong_augment = tf.keras.layers.GaussianNoise(stddev=5)

    self.cls_head = tf.keras.Sequential([
      tf.keras.layers.Dense(256,activation="relu"),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(64,activation="relu"),
      tf.keras.layers.Dense(self.num_classes, activation="softmax")
    ])

  def call(self, inputs, training):
    ids, mask = inputs
    
    embeds = self.bert(input_ids=ids, attention_mask=mask,training=training).pooler_output

    strongs = self.strong_augment(embeds,training=training)
    weaks = self.weak_augment(embeds,training=training)

    strong_preds = self.cls_head(strongs,training=training)
    weak_preds = self.cls_head(weaks,training=training)

    return weak_preds, strong_preds, strongs



In [15]:
proj_head = tf.keras.Sequential([
  tf.keras.layers.Dense(256,activation="relu"),
  tf.keras.layers.Dense(256)
])


In [16]:
model = FixMatchCR(bert_model=robert)

## Train and evaluate

In [17]:
optim = tfa.optimizers.AdamW(weight_decay=0.001,learning_rate=0.005)
optim_proj = tfa.optimizers.AdamW(weight_decay=0.000,learning_rate=0.001)
optim2 = tfa.optimizers.AdamW(weight_decay=0.0,learning_rate=0.00001)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
val_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
val_prec_metric = tf.keras.metrics.Precision(name="precision")
val_recall_metric = tf.keras.metrics.Recall(name="recall")
f1_metric_micro = tfa.metrics.F1Score(num_classes=4, threshold=0.5, average='micro', name='f1_micro')
f1_metric_macro = tfa.metrics.F1Score(num_classes=4, threshold=0.5, average='macro', name='f1_macro')

In [18]:
thresh = 0.7
thresh2 = 0.7
tau = 1.0
contrastive_weight = 1.0
unsup_weight = 1.0
num_classes=4
steps_per_epoch = 400

In [19]:
# @tf.function
# def train_step(x,y,xu):

#   with tf.GradientTape() as tape:
#       wl,_,_ = model([x["input_ids"],x["attention_mask"]],training=True)
#       wu,su,sembeds = model([xu["input_ids"],xu["attention_mask"]],training=True)
      
#       zu = proj_head(sembeds, training=True)

#       tu = tf.math.l2_normalize(zu,axis=1)

#       pseudo_labels = tf.reduce_max(wu,axis=1)
#       pseudo_idxs = tf.cast(tf.argmax(wu, axis=1),tf.int32)

#       sims = tf.exp(tu @ tf.transpose(tu) / tau)
#       cnt = sims.shape[0]
#       sims = tf.linalg.set_diag(sims, tf.zeros(cnt))
#       sums = tf.reshape(tf.reduce_sum(sims,axis=1),(-1,1))
#       ss = sims/sums
#       norm_sims = tf.where(tf.less(ss,1e-7), 1e-7,tf.math.log(ss))
#       norm_sims = tf.where(tf.math.is_nan(norm_sims),1e-7,norm_sims)
  
#       prods = tf.linalg.set_diag(norm_sims, tf.zeros(cnt))

#       a = tf.map_fn(lambda idx: tf.equal(pseudo_idxs,idx),tf.range(num_classes),fn_output_signature=bool)
#       mask1 = tf.linalg.set_diag(tf.gather(a,pseudo_idxs),tf.zeros(cnt,dtype=bool))

#       b = prods * tf.cast(mask1, tf.float32)

#       p_cardinal = tf.cast(tf.math.count_nonzero(mask1, axis=1),tf.float32)
      
#       r = -tf.reduce_sum(b,axis=1)/p_cardinal
#       r = tf.where(tf.math.is_nan(r),0.0,r)

#       mask2 = pseudo_labels>thresh2

#       lcr = tf.reduce_mean(r * tf.cast(mask2,float))
      
#       ls = loss_fn(y, wl)
#       mask = pseudo_labels>thresh
#       wu = wu[mask]
#       su = su[mask]
#       lu = loss_fn(tf.argmax(wu,axis=1),su)

#       tf.print(ls, lu, lcr)
#       tf.print(wl)

#       loss = ls + unsup_weight * lu + contrastive_weight * lcr

#   grads = tape.gradient(loss, [model.cls_head.trainable_weights, model.bert.trainable_weights, proj_head.trainable_weights])
#   tf.print(grads)
#   optim.apply_gradients(zip(grads[0], model.cls_head.trainable_weights))
#   optim2.apply_gradients(zip(grads[1], model.bert.trainable_weights))
#   optim_proj.apply_gradients(zip(grads[2], proj_head.trainable_weights))

#   return loss

@tf.function
def train_step(x,y,xu):

  with tf.GradientTape() as tape:
      wl,_,_ = model([x["input_ids"],x["attention_mask"]],training=True)
      wu,su,sembeds = model([xu["input_ids"],xu["attention_mask"]],training=True)
      
      zu = proj_head(sembeds, training=True)

      tu = tf.math.l2_normalize(zu,axis=1)

      cnt = tf.shape(wu)[0]
      pseudo_labels = tf.reduce_max(wu,axis=1)
      pseudo_idxs = tf.argmax(wu, axis=1)
      mask_labels = tf.cast(tf.equal(pseudo_idxs[:,tf.newaxis], pseudo_idxs[tf.newaxis,:]),tf.float32)

      sims = tf.exp(tu @ tf.transpose(tu) / tau)
      mask_self = tf.linalg.set_diag(tf.ones((cnt,cnt)),tf.zeros(cnt))
      sims = sims * mask_self
      sums = tf.reduce_sum(sims,axis=1,keepdims=True)
      sims = sims/sums
      sims = sims * mask_labels
      counts = tf.math.count_nonzero(sims, axis=1)
      mask_not_all_zero = counts > 0
      sims = sims[mask_not_all_zero]
      counts = tf.cast(counts[mask_not_all_zero],tf.float32)
      sims = tf.where(tf.equal(sims,0.0),1.0,sims)
      ru = -tf.reduce_sum(tf.math.log(sims),axis=1)/counts

      mask2 = pseudo_labels[mask_not_all_zero] > thresh2
      lcr = 0.0
      ru = ru[mask2]
      if tf.shape(ru)[0] != 0:
        lcr = tf.reduce_mean(ru[mask2])

      
      ls = loss_fn(y, wl)
      mask = pseudo_labels>thresh
      wu = wu[mask]
      su = su[mask]
      lu = loss_fn(tf.argmax(wu,axis=1),su)

      loss = ls + unsup_weight * lu + contrastive_weight * lcr

  grads = tape.gradient(loss, [model.cls_head.trainable_weights, model.bert.trainable_weights, proj_head.trainable_weights])
  optim.apply_gradients(zip(grads[0], model.cls_head.trainable_weights))
  optim2.apply_gradients(zip(grads[1], model.bert.trainable_weights))
  optim_proj.apply_gradients(zip(grads[2], proj_head.trainable_weights))

  return loss


@tf.function
def test_step(x, y):
    wpred,_,_ = model([x["input_ids"],x["attention_mask"]], training=False)
    val_acc_metric.update_state(y, wpred)
    true_hot = tf.one_hot(y, 4)
    val_prec_metric.update_state(true_hot, wpred)
    val_recall_metric.update_state(true_hot, wpred)
    f1_metric_micro.update_state(true_hot, wpred)
    f1_metric_macro.update_state(true_hot, wpred)

In [20]:
EPOCHS=8
max_val_acc = 0.0
best_weights = None

l_iter = iter(labeled_ds)
u_iter = iter(unlabeled_ds)
for epoch in range(EPOCHS):
  print(f"Epoch {epoch}")
  start_time = time.time()
  for step in range(steps_per_epoch):
    x, y = next(l_iter)
    xu = next(u_iter)
    
    loss = train_step(x,y,xu)
    if step % 100 == 0:
      print(
        "Training loss (for one batch) at step %d: %.4f"
        % (step, float(loss))
      )
  for x_batch_val, y_batch_val in val_ds:
      test_step(x_batch_val, y_batch_val)

  acc = float(val_acc_metric.result())
  prec = float(val_prec_metric.result())
  recall = float(val_recall_metric.result())
  micro = float(f1_metric_micro.result())
  macro = float(f1_metric_macro.result())

  val_acc_metric.reset_states()
  val_prec_metric.reset_states()
  val_recall_metric.reset_states()
  f1_metric_micro.reset_states()
  f1_metric_macro.reset_states()

  if acc > max_val_acc:
    max_val_acc = acc
    best_weights = model.get_weights()
  print(f"Validation acc: {acc:.3f} precision: {prec:.3f} recall: {recall:.3f} f1_micro: {micro:.3f} f1_macro: {macro:.3f}")
  print("Time taken: %.2fs" % (time.time() - start_time))
print("Restoring best weights relative to validation accuracy...")
model.set_weights(best_weights)

Epoch 0
Training loss (for one batch) at step 0: 3.6965
Training loss (for one batch) at step 100: 1.1331
Training loss (for one batch) at step 200: 1.2376
Training loss (for one batch) at step 300: 2.4034
Validation acc: 0.562 precision: 0.598 recall: 0.531 f1_micro: 0.563 f1_macro: 0.340
Time taken: 165.25s
Epoch 1
Training loss (for one batch) at step 0: 4.5221
Training loss (for one batch) at step 100: 3.7206
Training loss (for one batch) at step 200: 1.0152
Training loss (for one batch) at step 300: 0.6709
Validation acc: 0.651 precision: 0.762 recall: 0.501 f1_micro: 0.605 f1_macro: 0.381
Time taken: 151.27s
Epoch 2
Training loss (for one batch) at step 0: 1.8463
Training loss (for one batch) at step 100: 1.4258
Training loss (for one batch) at step 200: 1.6938
Training loss (for one batch) at step 300: 3.0405
Validation acc: 0.677 precision: 0.767 recall: 0.610 f1_micro: 0.680 f1_macro: 0.566
Time taken: 144.81s
Epoch 3
Training loss (for one batch) at step 0: 2.3851
Training lo

In [21]:
for x_batch_val, y_batch_val in test_ds:
    test_step(x_batch_val, y_batch_val)
acc = float(val_acc_metric.result())
prec = float(val_prec_metric.result())
recall = float(val_recall_metric.result())
micro = float(f1_metric_micro.result())
macro = float(f1_metric_macro.result())

val_acc_metric.reset_states()
val_prec_metric.reset_states()
val_recall_metric.reset_states()
f1_metric_micro.reset_states()
f1_metric_macro.reset_states()
print(f"Test acc: {acc:.3f} precision: {prec:.3f} recall: {recall:.3f} f1_micro: {micro:.3f} f1_macro: {macro:.3f}")

Test acc: 0.744 precision: 0.751 recall: 0.738 f1_micro: 0.745 f1_macro: 0.732


In [22]:
model.save_weights("./checkpoints/contrastive")